In [ ]:
!pip install sodapy
!pip install geopy
!pip install googlemaps
!pip install folium
import pandas as pd
from sodapy import Socrata
import googlemaps
from geopy.geocoders import Nominatim
import folium
from IPython.display import IFrame


# Configuración de no truncar visualización de los DF.
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [2]:
client = Socrata("www.datos.gov.co", None)
results = client.get("nfa3-wgxy", limit=50000)
results_df = pd.DataFrame.from_records(results)

In [3]:
results_df.columns

Index(['a_o_accidente', 'mes_accidente', 'clase_accidente',
       'servicio_vehiculo_accidentado', 'clase_vehiculo_accidentado',
       'cantidad_vehiculos', 'direccion_accidente'],
      dtype='object')

In [4]:
len(results_df)

43394

In [5]:
sum(results_df['a_o_accidente'] == '2023')

1986

In [6]:
df_2023 = results_df[results_df["a_o_accidente"] == "2023"]

In [7]:
len(df_2023)

1986

In [8]:
df_2023.iloc[6]

a_o_accidente                                                        2023
mes_accidente                                                           1
clase_accidente                                                    Choque
servicio_vehiculo_accidentado                                  PARTICULAR
clase_vehiculo_accidentado                                    MOTOCICLETA
cantidad_vehiculos                                                      1
direccion_accidente              AV. CIRCUNVALAR CRA 68 VENTANA DEL MUNDO
Name: 41414, dtype: object

In [9]:
df_2023.tail()

,a_o_accidente,mes_accidente,clase_accidente,servicio_vehiculo_accidentado,clase_vehiculo_accidentado,cantidad_vehiculos,direccion_accidente
43389,2023,8,Choque,PARTICULAR,CAMIONETA,1,VIA 40 CON 76
43390,2023,8,Choque,PARTICULAR,AUTOMOVIL,1,VIA 40 CON CALLE 52 - 332
43391,2023,8,Choque,PARTICULAR,MOTOCICLETA,1,VIA 40 CON CALLE 52 - 332
43392,2023,8,Choque,PUBLICO,BUS,1,VIA LA PLAYA - COLEGIO PIES DESCALZOS
43393,2023,8,Choque,(en blanco),BICICLETA,1,VIA LA PLAYA - COLEGIO PIES DESCALZOS


In [11]:
def obtener_latitud_longitud_geopy(direccion):
    geolocator = Nominatim(user_agent="my_geocoder")
    try:
        location = geolocator.geocode(direccion, timeout=10)
    except Exception as e:
        print(f"Error en la solicitud Geopy: {e}")
        location = None

    if location:
        latitud = location.latitude
        longitud = location.longitude
        return latitud, longitud
    else:
        return None, None


def obtener_latitud_longitud_google(direccion):
    gmaps = googlemaps.Client(key='API_Key')
    resultado = gmaps.geocode(direccion)

    if resultado:
        latitud = resultado[0]['geometry']['location']['lat']
        longitud = resultado[0]['geometry']['location']['lng']
        return latitud, longitud
    else:
        return None, None

def obtener_latitud_longitud(direccion):
    latitud, longitud = obtener_latitud_longitud_geopy(direccion)

    if latitud is None or longitud is None:
        latitud, longitud = obtener_latitud_longitud_google(direccion)

    return latitud, longitud

In [12]:
latlong = []
# Iterar sobre las direcciones y obtener latitud y longitud
for direccion in df_2023['direccion_accidente']:
    latitud, longitud = obtener_latitud_longitud_google(direccion)
    latlong.append({'Latitud': latitud, 'Longitud': longitud})

In [45]:
df_clean_accidentes_2023 = pd.read_csv("..\\Escritorio\\lat_long_accidentes_bquilla")

In [46]:
len(df_clean_accidentes_2023)

1986

In [47]:
df_clean_accidentes_2023.dropna(inplace=True)

In [48]:
len(df_clean_accidentes_2023)

1367

In [49]:
# Obtener el promedio de las coordenadas para centrar el mapa
promedio_latitud = df_clean_accidentes_2023['Latitud'].mean()
promedio_longitud = df_clean_accidentes_2023['Longitud'].mean()

# Crea un objeto gmplot centrado en las coordenadas promedio
barranquilla_map = folium.Map(location=[promedio_latitud, promedio_longitud], zoom_start=13)

# Agrega marcadores para cada accidente en el DataFrame
for index, row in df_clean_accidentes_2023.iterrows():
    latitud = row['Latitud']
    longitud = row['Longitud']
    folium.Marker(location=[latitud, longitud]).add_to(barranquilla_map)


barranquilla_map
